In [1]:
import pickle
import numpy as np
import random
import glob
import matplotlib.pylab as plt
import numpy as np
from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as func
from torchvision.models import resnet18
from torchvision import transforms, models
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from scipy.stats import entropy
from getData import *
from trainer import *
from getModel import *
#import torchprofile

import time
# Seed for reproducibility
SEED = 25
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

C:\Users\rafor\anaconda3\envs\endg511\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


cuda:0


# **Load Dataset**

In [2]:
dataset = TrainValDataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size= 32, shuffle = True, num_workers= 0)

val_loader = DataLoader(val_dataset, batch_size= 32, shuffle = False, num_workers= 0)

test_dataset = TestDataset()

test_loader = DataLoader(test_dataset, batch_size= 32, shuffle = False, num_workers= 0)


['Training\\Fire', 'Training\\No_Fire']
['Test\\Fire', 'Test\\No_Fire']


# **Baseline Model**

In [3]:
NUM_EPOCHS = 10

In [4]:
blNet = blModel().to(device)
optimizer_bl = optim.Adam(blNet.parameters(), lr=1e-3)
scheduler = None
criterion_bl = nn.CrossEntropyLoss()

In [5]:
bl_titleToday = "./Models/best_bl_0319.pth"
BL = blHandler(blNet, criterion_bl, optimizer_bl, device=device, scheduler=scheduler, num_epochs=NUM_EPOCHS, bestPath=bl_titleToday)
blNet, blHistory = BL.train(train_loader, test_loader)

epoch 1 --> trainLoss: 0.336, trainAcc: 0.978, validLoss: 0.741, validAcc: 0.565
Model Saved!
epoch 2 --> trainLoss: 0.325, trainAcc: 0.988, validLoss: 0.715, validAcc: 0.595
Model Saved!
epoch 3 --> trainLoss: 0.322, trainAcc: 0.991, validLoss: 0.732, validAcc: 0.576
epoch 4 --> trainLoss: 0.321, trainAcc: 0.992, validLoss: 0.733, validAcc: 0.578
epoch 5 --> trainLoss: 0.320, trainAcc: 0.993, validLoss: 0.636, validAcc: 0.673
Model Saved!
epoch 6 --> trainLoss: 0.320, trainAcc: 0.993, validLoss: 0.715, validAcc: 0.590
epoch 7 --> trainLoss: 0.318, trainAcc: 0.995, validLoss: 0.703, validAcc: 0.604
epoch 8 --> trainLoss: 0.319, trainAcc: 0.994, validLoss: 0.691, validAcc: 0.619
epoch 9 --> trainLoss: 0.318, trainAcc: 0.995, validLoss: 0.712, validAcc: 0.594
epoch 10 --> trainLoss: 0.319, trainAcc: 0.994, validLoss: 0.733, validAcc: 0.575


In [5]:
bl_titleToday = "./Models/best_bl_0319.pth"
blNet.load_state_dict(torch.load("./Models/best_bl_0319.pth"))
BL = blHandler(blNet, criterion_bl, optimizer_bl, device=device, scheduler=scheduler, num_epochs=NUM_EPOCHS, bestPath=bl_titleToday)
startTime = time.time()
predicted, acc = BL.infer(test_loader)
executionTime= (time.time() - startTime)/len(test_dataset)

In [6]:
print(acc)
print(executionTime)

0.6734953703703703
0.0014931765208045665


# **EE Model**

In [11]:
dataset = TrainValDataset()
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size= 32, shuffle = True, num_workers= 0)

val_loader = DataLoader(val_dataset, batch_size= 32, shuffle = False, num_workers= 0)

test_dataset = TestDataset()

test_loader = DataLoader(test_dataset, batch_size= 32, shuffle = False, num_workers= 0)

['Training\\Fire', 'Training\\No_Fire']
['Test\\Fire', 'Test\\No_Fire']


In [12]:
NUM_EPOCHS = 10

In [13]:
eeNet_V0 = eeModel_V0().to(device)
optimizer_ee_V0 = optim.Adam(eeNet_V0.parameters(), lr=1e-3)
scheduler_V0 = None
criterion_ee_V0 = nn.CrossEntropyLoss()
V0_titleToday = "./Models/best_eev0.pth"
EE_V0 = eeHandler(eeNet_V0, criterion_ee_V0, optimizer_ee_V0, device=device, scheduler=scheduler_V0, num_epochs=NUM_EPOCHS, bestPath=V0_titleToday)

Number of base parameters: 1188
Number of short branch parameters: 6952450
Number of long branch parameters: 13380882
Difference = 6428432


In [14]:
eeNetV0, eeV0History = EE_V0.train(train_loader, test_loader)

epoch 1 --> trainLoss: 0.336, trainAcc: 0.977, validLoss: 0.764, validAcc: 0.541
Model Saved!
epoch 2 --> trainLoss: 0.323, trainAcc: 0.990, validLoss: 0.736, validAcc: 0.570
Model Saved!
epoch 3 --> trainLoss: 0.321, trainAcc: 0.992, validLoss: 0.784, validAcc: 0.520
epoch 4 --> trainLoss: 0.319, trainAcc: 0.994, validLoss: 0.798, validAcc: 0.500
epoch 5 --> trainLoss: 0.319, trainAcc: 0.994, validLoss: 0.738, validAcc: 0.569
epoch 6 --> trainLoss: 0.318, trainAcc: 0.995, validLoss: 0.741, validAcc: 0.565
epoch 7 --> trainLoss: 0.318, trainAcc: 0.995, validLoss: 0.747, validAcc: 0.558
epoch 8 --> trainLoss: 0.318, trainAcc: 0.996, validLoss: 0.726, validAcc: 0.578
Model Saved!
epoch 9 --> trainLoss: 0.318, trainAcc: 0.996, validLoss: 0.788, validAcc: 0.518
epoch 10 --> trainLoss: 0.317, trainAcc: 0.996, validLoss: 0.752, validAcc: 0.550


In [15]:
V0_titleToday = "./Models/best_eev0.pth"
eeNet_V0.load_state_dict(torch.load("./Models/best_eev0.pth"))
EE_V0 = eeHandler(eeNet_V0, criterion_ee_V0, optimizer_ee_V0, device=device, scheduler=scheduler_V0, num_epochs=NUM_EPOCHS, bestPath=V0_titleToday)
startTime = time.time()
rec, pred, acc = EE_V0.infer(test_loader, threshold = 0.1)
executionTime= (time.time() - startTime)/len(test_dataset)

In [16]:
print(acc)
print(executionTime)

0.5847742833932923
0.0026283626892844194


In [17]:
EE_V0.testingSummary(rec)

Summary
Branch 1: Accuracy 58.90% with 86.67% of the samples
Branch 2: Accuracy 55.70% with 13.33% of the samples
Overall Weighted Accuracy: 58.48%
